## 0. Requirements

To run this notebook, start jupyter notebook with the ae483 environment active (see [MacOS](https://canvas.illinois.edu/courses/7116/modules/items/1230512) or [Windows](https://canvas.illinois.edu/courses/7116/modules/items/1270318) instructions for how to create this environment). You will also need to add [ipywidgets](https://ipywidgets.readthedocs.io/en/stable/user_install.html) to this environment from a terminal (MacOS) or an anaconda powershell (Windows) before starting jupyter notebook, as follows:
```
pip install ipywidgets
```
You will, of course, only need to do this once.

## 1. Set up the notebook

Import all the modules we need.

In [ ]:
import os
import json
import time
import numpy as np
import sympy as sym
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
import ipywidgets as widgets
from ipywidgets import interact
from IPython.display import clear_output
from scipy.integrate import solve_ivp

# This is my own script (it is an interface to the pybullet simulator)
import ae483_drone

# I often go back and forth between making changes to my scripts and to
# the notebook in which they are used. One "gotcha" is that notebooks only
# import modules or scripts ONCE. Subsequent imports don't do anything, and
# in particular won't reflect any changes I've made to my scripts. To make
# sure that I'm working with the latest version of my code, I use this bit
# of magic, which forces the notebook to "reload" my script:
import importlib
importlib.reload(ae483_drone)

Start the simulator.

In [ ]:
simulator = ae483_drone.Simulator(width=800, height=600)

## 2. Visualize the ZYX (yaw, pitch, roll) Euler angle sequence

Create a widget that allows us to change the position and orientation of the drone. Re-evaluate this cell to reset the position and orientation.

In [ ]:
@interact
def set_pose(
    x=(-1., 1., 0.01),
    y=(-1., 1., 0.01),
    z=(-1., 1., 0.01),
    yaw=(-3.2, 3.2, 0.05),
    pitch=(-3.2, 3.2, 0.05),
    roll=(-3.2, 3.2, 0.05)):
    simulator.set_pose(x, y, z, yaw, pitch, roll)

Same thing in degrees.

In [ ]:
@interact
def set_pose(
    x=(-1., 1., 0.01),
    y=(-1., 1., 0.01),
    z=(-1., 1., 0.01),
    yaw_deg=(-180, 180, 1.),
    pitch_deg=(-180, 180, 1.),
    roll_deg=(-180, 180, 1.)):
    simulator.set_pose(x, y, z, np.deg2rad(yaw_deg), np.deg2rad(pitch_deg), np.deg2rad(roll_deg))

## 3. Compute the rotation matrix produced by yaw, pitch, roll

Define symbolic variables.

In [ ]:
psi, theta, phi = sym.symbols('psi, theta, phi')

Define individual rotation matrices.

In [ ]:
Rz = sym.Matrix([[sym.cos(psi), -sym.sin(psi), 0],
                 [sym.sin(psi), sym.cos(psi), 0],
                 [0, 0, 1]])

Ry = sym.Matrix([[sym.cos(theta), 0, sym.sin(theta)],
                 [0, 1, 0],
                 [-sym.sin(theta), 0, sym.cos(theta)]])

Rx = sym.Matrix([[1, 0, 0],
                 [0, sym.cos(phi), -sym.sin(phi)],
                 [0, sym.sin(phi), sym.cos(phi)]])

Show each rotation matrix.

In [ ]:
Rz

In [ ]:
Ry

In [ ]:
Rx

Apply sequential transformation to compute the rotation matrix that describes the orientation of the drone (i.e., of frame 1 in the coordinates of frame 0).

In [ ]:
R_1in0 = Rz * Ry * Rx

Show this rotation matrix.

In [ ]:
R_1in0

Create a function that returns this rotation matrix as a numpy array.

In [ ]:
R = sym.lambdify((psi, theta, phi), R_1in0)

Evaluate this function at particular angles (for example).

In [ ]:
print(R(0., 0., 0.))

Create a widget that allows us to change the position and orientation of the drone, *while also displaying the position and orientation in coordinates*. Re-evaluate this cell to reset the position and orientation.

In [ ]:
@interact
def set_pose(
    x=(-1., 1., 0.01),
    y=(-1., 1., 0.01),
    z=(-1., 1., 0.01),
    yaw=(-3.2, 3.2, 0.05),
    pitch=(-3.2, 3.2, 0.05),
    roll=(-3.2, 3.2, 0.05)):
    simulator.set_pose(x, y, z, yaw, pitch, roll)
    
    o_string = np.array2string(np.array([[x], [y], [z]]),
                               formatter={'float_kind': lambda x: f'{x:7.4f}'},
                               prefix='    ')
    R_string = np.array2string(R(yaw, pitch, roll),
                               formatter={'float_kind': lambda x: f'{x:7.4f}'},
                               prefix='    ')
    
    print(f'position:\n\no = {o_string}')
    print('')
    print(f'orientation:\n\nR = {R_string}')
    

Same thing in degrees.

In [ ]:
@interact
def set_pose(
    x=(-1., 1., 0.01),
    y=(-1., 1., 0.01),
    z=(-1., 1., 0.01),
    yaw_deg=(-180, 180, 1.),
    pitch_deg=(-180, 180, 1.),
    roll_deg=(-180, 180, 1.)):
    
    yaw = np.deg2rad(yaw_deg)
    pitch = np.deg2rad(pitch_deg)
    roll = np.deg2rad(roll_deg)
    
    simulator.set_pose(x, y, z, yaw, pitch, roll)
    
    o_string = np.array2string(np.array([[x], [y], [z]]),
                               formatter={'float_kind': lambda x: f'{x:7.4f}'},
                               prefix='    ')
    R_string = np.array2string(R(yaw, pitch, roll),
                               formatter={'float_kind': lambda x: f'{x:7.4f}'},
                               prefix='    ')
    
    print(f'position:\n\no = {o_string}')
    print('')
    print(f'orientation:\n\nR = {R_string}')

### Is "yaw, pitch, roll" the same as "roll, pitch, yaw"?

Here is the rotation matrix produced by a "yaw, pitch, roll" sequence:

In [ ]:
Rz * Ry * Rx

Here is the rotation matrix produced by a "roll, pitch, yaw" sequence:

In [ ]:
Rx * Ry * Rz

Are they the same? If not, then the order of axes about which you rotate matters.

## 4. Compute the matrix that relates angular velocity and angular rates

Recall that

$$\begin{bmatrix} \dot{\psi} \\ \dot{\theta} \\ \dot{\phi} \end{bmatrix} = N w_{0, 1}^{1}$$

for some matrix $N$. Here is how to compute that matrix for a ZYX Euler angle sequence.  First, we compute its inverse:

In [ ]:
Ninv = sym.Matrix.hstack((Ry * Rx).T * sym.Matrix([[0], [0], [1]]),
                              (Rx).T * sym.Matrix([[0], [1], [0]]),
                                       sym.Matrix([[1], [0], [0]]))

Show $N^{-1}$:

In [ ]:
Ninv

Then, we compute $N$ by taking the inverse of $N^{-1}$:

In [ ]:
N = sym.simplify(Ninv.inv())

Show $N$:

In [ ]:
N

Create a function that returns this matrix as a numpy array:

In [ ]:
N = sym.lambdify((psi, theta, phi), N)

## 5. Visualize angular motion

The following ODEs describe the kinematics of the drone:

$$ \begin{align*} \dot{o}_1^0 &= R_1^0 v_{0, 1}^{1} \\ \begin{bmatrix} \dot{\psi} \\ \dot{\theta} \\ \dot{\phi} \end{bmatrix} &= N w_{0, 1}^1 \end{align*}$$

Define a function that solves these ODEs over some time interval to update the position and orientation, assuming linear and angular velocity are constant.

In [ ]:
def update_pose(pose, v_01in1, w_01in1, dt):
    """
    Given body-fixed linear and angular velocity, this function integrates
    the kinematics to compute the position and orientation after dt seconds.
    """
    sol = solve_ivp(lambda t, y: np.concatenate((R(y[3], y[4], y[5]) @ v_01in1,
                                                 N(y[3], y[4], y[5]) @ w_01in1)), # <-- the ODEs
                    [0., dt], # <-- the time interval over which to integrate
                    np.array([pose['o_x'],
                              pose['o_y'],
                              pose['o_z'],
                              pose['psi'],
                              pose['theta'],
                              pose['phi']]), # <-- the initial condition
                    t_eval=[dt]) # <-- the time at which to return a solution
    y = sol.y.flatten()
    pose['o_x'] = y[0]
    pose['o_y'] = y[1]
    pose['o_z'] = y[2]
    pose['psi'] = y[3]
    pose['theta'] = y[4]
    pose['phi'] = y[5]
    

Define a function to print position, orientation, angular velocity, and angular rates.

In [ ]:
def print_motion(pose, w_01in1):
    o_string = np.array2string(np.array([[pose['o_x']], [pose['o_y']], [pose['o_z']]]),
                               formatter={'float_kind': lambda x: f'{x:7.4f}'},
                               prefix='    ')
    R_string = np.array2string(R(pose['psi'], pose['theta'], pose['phi']),
                               formatter={'float_kind': lambda x: f'{x:7.4f}'},
                               prefix='    ')
    a_string = np.array2string(np.array([pose['psi'], pose['theta'], pose['phi']]),
                               formatter={'float_kind': lambda x: f'{x:7.4f}'})
    w_string = np.array2string(w_01in1,
                               formatter={'float_kind': lambda x: f'{x:10.4f}'})
    rates_string = np.array2string(N(pose['psi'], pose['theta'], pose['phi']) @ w_01in1,
                               formatter={'float_kind': lambda x: f'{x:10.4f}'})

    print(f'position:\n\no = {o_string}' +
          '\n\n' +
          f'orientation:\n\nR = {R_string}\n\n[psi, theta, phi] = {a_string}' + 
          '\n\n' +
          f'angular velocity:\n\nw_01in1 = {w_string}' +
          '\n\n' +
          f'angular rates:\n\n[psidot, thetadot, phidot] = {rates_string}'
         )

Define a function to visualize the resulting motion.

In [ ]:
def visualize_motion(pose, v_01in1, w_01in1, max_time, sleep_time=1.):
    # Time step
    dt = 0.05

    # Index of current time step
    cur_step = 0

    # Index of time step at which to stop the motion
    max_step = int(max_time / dt)

    # Set initial pose
    simulator.set_pose(pose['o_x'],
                       pose['o_y'],
                       pose['o_z'],
                       pose['psi'],
                       pose['theta'],
                       pose['phi'])
    
    # Show position, orientation, angular velocity, and angular rates
    print_motion(pose, w_01in1)
    
    # Pause for a moment
    time.sleep(sleep_time)
    
    # Loop until done
    start_time = time.time()
    while cur_step < max_step:
        # Increment the index of time step
        cur_step += 1

        # Integrate to update position and orientation
        update_pose(pose, v_01in1, w_01in1, dt)

        # Wait, attempting to visualize the motion in real time
        while time.time() - start_time < cur_step * dt:
            time.sleep(0.01)

        # Set current pose
        simulator.set_pose(pose['o_x'],
                       pose['o_y'],
                       pose['o_z'],
                       pose['psi'],
                       pose['theta'],
                       pose['phi'])

        # Clear notebook output
        clear_output(wait=True)

        # Show position, orientation, angular velocity, and angular rates
        print_motion(pose, w_01in1)

Visualize an example motion.

In [ ]:
# Initial position and orientation
pose = {
    'o_x': 0.,
    'o_y': 0.,
    'o_z': 0.3,
    'psi': 0.,
    'theta': 0.,
    'phi': 0.,
}

# Linear velocity
v_01in1 = np.array([0., 0., 0.])

# Angular velocity
w_01in1 = np.array([0., 0., 0.])

visualize_motion(pose, v_01in1, w_01in1, 10.)

Another example. Pay close attention to the angular rates.

In [ ]:
# Initial position and orientation
pose = {
    'o_x': 0.,
    'o_y': 0.,
    'o_z': 0.3,
    'psi': 0.,
    'theta': 0.,
    'phi': 0.01,
}

# Linear velocity
v_01in1 = np.array([0., 0., 0.])

# Angular velocity
w_01in1 = np.array([0., 1., 0.])

visualize_motion(pose, v_01in1, w_01in1, 2 * np.pi)